<a href="https://colab.research.google.com/github/JieUpup/6220-Homework2/blob/main/jzrepo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import os

!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 21.2 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=47324e247eed8960f112ba909a24e9b68e689427e1c4c3ee718e1b6641911f7c
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark
The following package was automatically installed and is no longer required:
  libnvidia-common-510
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The followin

In [5]:
#@title Create a Spark Session and Context

from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import pandas as pd

# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [12]:
path = "soc-LiveJournal1Adj.txt"
df1 = spark.read.text(path)
df1.show()

AnalysisException: ignored

In [9]:
# small data demo and display
sample_data = open("small_sample.txt", "w")  # append mode
sample_data.write("1\t2,3,5\n")
sample_data.write("2\t1,3,5,6\n")
sample_data.write("3\t1,2,7\n")
sample_data.write("4\t5,6,7\n")
sample_data.write("5\t1,2,4\n")
sample_data.write("6\t2,4,7\n")
sample_data.write("7\t3,4,6\n")
sample_data.close()

df1 = spark.read.text("small_sample.txt")
df1.show()

+----------+
|     value|
+----------+
|  1\t2,3,5|
|2\t1,3,5,6|
|  3\t1,2,7|
|  4\t5,6,7|
|  5\t1,2,4|
|  6\t2,4,7|
|  7\t3,4,6|
+----------+



In [14]:
# formart the data frame
df2 = spark.read.option("delimiter", "\t").csv("small_sample.txt")
df2.show()

+---+-------+
|_c0|    _c1|
+---+-------+
|  1|  2,3,5|
|  2|1,3,5,6|
|  3|  1,2,7|
|  4|  5,6,7|
|  5|  1,2,4|
|  6|  2,4,7|
|  7|  3,4,6|
+---+-------+



In [15]:
df2 = df2.withColumnRenamed('_c0', 'user_id')
df2 = df2.withColumnRenamed('_c1', 'friends')
df2.show()

+-------+-------+
|user_id|friends|
+-------+-------+
|      1|  2,3,5|
|      2|1,3,5,6|
|      3|  1,2,7|
|      4|  5,6,7|
|      5|  1,2,4|
|      6|  2,4,7|
|      7|  3,4,6|
+-------+-------+



In [16]:
# define a function to get the id by "friends"
def get_main_id(df, main_id):
    df = df.filter(df.user_id == str(main_id))
    r1 = df.select('friends').collect()[0]

    return (str(main_id), r1.__getitem__('friends'))

# define a fucntion to find the common friends, use two parameters f1 and f2

def common(main_id, cur_id, f1, f2):
  # use current_id to track the main_id
  if main_id == cur_id:
    return 0

  arr1 = f1.split(',') if f1 != None else []
  arr2 = f2.split(',') if f2 != None else []
  if main_id in arr2:
    return 0
  else:
    # use sort and track the index of array1 and array2, use viriable count
    # to update the interator
    arr1.sort()
    arr2.sort()
    index1 = 0
    index2 = 0
    count = 0
    while index1 < len(arr1) and index2 < len(arr2):
      if arr1[index1] == arr2[index2]:
        count += 1
        index1 += 1
        index2 += 1
      elif arr1[index1] > arr2[index2]:
        index2 += 1
      else:
        index1 += 1
    return count

main_id = '2'
main_id, main_friends = get_main_id(df2, main_id)
print(common(main_id, '2', main_friends, '3,4,5'))

0


In [ ]:
# define top_10 recomendation for user id.
def top_10(df, main_id, main_friends):
  # use inline lamda functon to mapping the result and filter.
  df_tu = df.rdd.map(lambda f : (int(f[0]), common(main_id, f[0], main_friends, f[1]))) \
      .filter(lambda f : f[1] > 0)
  if df_tu.isEmpty():
    return ""
  df2 = df_tu.toDF(['user_id', 'common_friends'])
  top_10_list = df2.sort(desc('common_friends'), asc('user_id')).select(col('user_id')).take(10)
  # use getitem method by user id and make sure it already exist in the previous list
  top_10_list = [str(e.__getitem__('user_id')) for e in top_10_list]
  friend_suggestions = ",".join(top_10_list)

  return friend_suggestions

#new_rdd = top_10(df2, '11')
#print(new_rdd)

li = df2.rdd.take(1000)

#df2_ddl = sc.parallelize(li)
#df3_ddl = df2.rdd.map(lambda x : (x[0], top_10(df2, x[0], x[1])))
#print(df3_ddl)
#print(type(li))

output = []
for user in li:
  user_id = user['user_id']
  friends = user['friends']
  suggestions = top_10(df2, user_id, friends)
  output.append( str(user_id) + '\t' + suggestions )


with open('output.text', 'w') as txt_file:
  for line in output:
    txt_file.write(line + '\n')


In [ ]:
!more output.text

1	4,6,7
2	4,7
3	5,6,4
4	2,1,3
5	3,6,7
6	3,5,1
7	2,1,5
